In [23]:
# Name: Thomas Scott
# Unit 19: Blockchain Wallet
# Late submission

# Import libraries
import subprocess
import json
import os
from constants import *
from dotenv import load_dotenv
from bipwallet import wallet
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

load_dotenv()

True

In [24]:
# import your mnemonic key from your local env file

mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

novel foot access crisp photo quiz suspect obey loud village speed exile


In [25]:
# This will connect you to your local ETH testnet 

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [27]:
# This function will derive the new wallets using hd-wallet-derive and PHP

def derive_wallets(mnemonic, coin, numderive):
    
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    
    return  keys

In [28]:
# Next we will test our function to make sure it gives us the output we are looking for

derive_wallets(mnemonic, 'BTC', 3)

[{'path': "m/44'/0'/0'/0/0",
  'address': '16ZxGx4dQ9Gu5gNHSTHChHZhkt9ZZZCazi',
  'xprv': 'xprvA4BysqMVf7kyTHveMZhMfyy71M7Cu1e65yyZwikqzzSqhcKDGKbwy7JA2agLwAopGo47HGtMjRtw1HDqvPcqwnrBmpjtFEb3k1J8qLQJfeT',
  'xpub': 'xpub6HBLHLtPVVKGfn17TbEN37uqZNwhJUMwTCuAk7ATZKypaQeMorvCWucdspNdiTsgvyUNkzepQAXXDWdSsqhtXA5NxhtR73sudoAESEyhWWb',
  'privkey': 'L4kvRAmvKEF6HZ6PfrRp3T2nWzijhmuUADEExAFDjm35KwnJ3G81',
  'pubkey': '0230ff3dcc3a656fabf91d0013860ae5c94333e495f75c0540c9864c76a2a06b26',
  'pubkeyhash': '3d1414b809aae8a4504532f763cd243fb41c1d1c',
  'index': 0},
 {'path': "m/44'/0'/0'/0/1",
  'address': '14nfeZNEt1Cwudca6J8ZgHWi4EpyDCGmXH',
  'xprv': 'xprvA4BysqMVf7kyVuprZ8PZxQHwzACWzKcQkzmHw58reej2KmNmCmy19P5gCWABrhKYT2k5WUGx3YjysUwiLLrQL8jCf4Fmo6p3t9q3knqjySU',
  'xpub': 'xpub6HBLHLtPVVKGiPuKf9vaKYEgYC31PnLG8DgtjTYUCzG1CZhukKHFhBQA3nSAK6WTgWvur9F8RStZT7SRUCBaFKXFqRWKMWNWefxm1tfjoCv',
  'privkey': 'L2CSPpCKTjNwRNgc93eZiope1koTZMaYvK23hT6xfekNfwunLrzW',
  'pubkey': '03387cd314b5c3fc90aa8a030d88349f

In [29]:
# This function will essentially objectify the 'privkey' string into an account object.

def priv_key_to_account(coin,priv_key):
    
    print(coin)
    
    print(priv_key)
    
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [31]:
# Define the function that generates the raw, unsigned transaction.

def create_tx(coin,account, recipient, amount):

    # This part is obviously for etherium
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    # This handles when the coin is of type btc or btc-test
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [32]:
# Next we will create a variable to hold our ETH private key

eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH,5)[0]['privkey'])

eth
0xba9dafd1b2c0d856fb4200d3a274b2c62edfba1b6bfba405f7ac7dc59a44adb0


In [33]:
# This function will create a transaction, sign the it, then send it to the designated network.

def send_tx(coin,account,recipient, amount):
    
    txn = create_tx(coin, account, recipient, amount)
    
    # Ether logic
    if coin == ETH:
        
        signed_txn = eth_acc.sign_transaction(txn)
        
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        
        print(result.hex())
        
        return result.hex()
    
    # Logic for BTC or BTC-Test
    elif coin == BTCTEST:
        
        tx_btctest = create_tx(coin, account, recipient, amount)
        
        signed_txn = account.sign_transaction(txn)
        
        print(signed_txn)
        
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [89]:
# BTC TEST

In [35]:
# Initialize constants. I saw no need to call them from the python file. I suppose it is more modular the other way.

coins = {"eth", "btc-test", "btc"}

numderive = 3

In [36]:
# Create a dictionary of keys.

keys = {}

#  Populate them with accounts encompassing the different coin types I have specified.
for coin in coins:
    keys[coin]= derive_wallets(mnemonic, coin, numderive)

In [ ]:
# Generate private keys for bitcoin and ether

eth_PrivKey = keys["eth"][0]['privkey']

btc_PrivKey = keys['btc-test'][0]['privkey']

In [37]:
# Print out the private keys for both

print(json.dumps(eth_PriveKey, indent=4, sort_keys=True))

print(json.dumps(btc_PrivKey, indent=4, sort_keys=True))

"0xba9dafd1b2c0d856fb4200d3a274b2c62edfba1b6bfba405f7ac7dc59a44adb0"
"cSRb3ZvfsjZo123dXkKRTbaYFCoF5ZBgyHAEULiAjXpwKHNjxDWK"


In [38]:
# Show all accounts with their addresses and keys
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "16ZxGx4dQ9Gu5gNHSTHChHZhkt9ZZZCazi",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "L4kvRAmvKEF6HZ6PfrRp3T2nWzijhmuUADEExAFDjm35KwnJ3G81",
            "pubkey": "0230ff3dcc3a656fabf91d0013860ae5c94333e495f75c0540c9864c76a2a06b26",
            "pubkeyhash": "3d1414b809aae8a4504532f763cd243fb41c1d1c",
            "xprv": "xprvA4BysqMVf7kyTHveMZhMfyy71M7Cu1e65yyZwikqzzSqhcKDGKbwy7JA2agLwAopGo47HGtMjRtw1HDqvPcqwnrBmpjtFEb3k1J8qLQJfeT",
            "xpub": "xpub6HBLHLtPVVKGfn17TbEN37uqZNwhJUMwTCuAk7ATZKypaQeMorvCWucdspNdiTsgvyUNkzepQAXXDWdSsqhtXA5NxhtR73sudoAESEyhWWb"
        },
        {
            "address": "14nfeZNEt1Cwudca6J8ZgHWi4EpyDCGmXH",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L2CSPpCKTjNwRNgc93eZiope1koTZMaYvK23hT6xfekNfwunLrzW",
            "pubkey": "03387cd314b5c3fc90aa8a030d88349f1e8814a27ad9e9de643908767c7fa43809",
            "pubkeyhash":

In [39]:
# Initialize a btc private key

btc_acc = priv_key_to_account(BTCTEST,btc_PrivKey)

btc-test
cSRb3ZvfsjZo123dXkKRTbaYFCoF5ZBgyHAEULiAjXpwKHNjxDWK


In [44]:
# Create a sample transaction using funds from a bitcoin faucet
# NOTE: You must fill up for it to work (https://testnet-faucet.mempool.co/)

create_tx(BTCTEST,btc_acc,"mtWmTgmtbTCKtaxeks4Gh58WgwpCjv6mAQ", 0.0005)

'{"unspents":[{"amount":100000,"confirmations":1,"script":"76a9148e90976baf3f8ea0c7728c7a9723eaf562d50e6888ac","txid":"ed218c866715ac20fa9b8c963f535406046709d74c9c7a336ea23493dd9f9848","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mtWmTgmtbTCKtaxeks4Gh58WgwpCjv6mAQ",50000],["mtWmTgmtbTCKtaxeks4Gh58WgwpCjv6mAQ",26948]]}'

In [45]:
# shift your coins between btc accounts

send_tx(BTCTEST,btc_acc,"mkk5aqK2ZqueLN1WbLjMEQgBwc4HiWnqtM", 0.0005)

010000000148989fdd9334a26e337a9c4cd70967040654533f968c9bfa20ac1567868c21ed010000006a473044022062f04170045465b9e5236b130c43d22188b8a27b0b31b8361355e90030c59ee202200e16bf5cccca2ba487328da698d322b0a56c6a92760505c2b9c7b9800cd6d21301210310f72ca09b7b5c95b30500ee3a28ccf604140926ebce57cfc9219d724818a292ffffffff0250c30000000000001976a91439541f08d0bfe43951a0ba429435f530340b7c1488ac44690000000000001976a9148e90976baf3f8ea0c7728c7a9723eaf562d50e6888ac00000000


In [46]:
# ETH Tests

In [47]:
# Connect to your ether testnet
# Note: I reused my testnet from the last assignment for the sake of time.

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0xba9dafd1b2c0d856fb4200d3a274b2c62edfba1b6bfba405f7ac7dc59a44adb0"))


In [48]:
# Return a boolean to signify whether we are connected to your local testnet
w3.isConnected()


True

In [68]:
# Return the balance for your primary ETH wallet
# Note: I topped off this wallet with coins mines from the last assignment. I couldn't get the instructor's web.py 
# alteration to work.

w3.eth.getBalance("0xf34A0b44dD795a72d5863E576595cA3C9853f44a")


180925139433306555349329664076074856020734351000000000000000000000

In [84]:
# Send a coins to an empty account in your testnet. 
# Note: I reused the wallet I emptied at the end of the last assignment for efficiency

create_tx(ETH, eth_acc,"0x268C8620F9a994410040D9B6DBE1E203451542c8", 18092513943330655534932)

{'from': '0xf34A0b44dD795a72d5863E576595cA3C9853f44a',
 'to': '0x268C8620F9a994410040D9B6DBE1E203451542c8',
 'value': 18092513943330655534932,
 'gasPrice': 60000000000,
 'gas': 21000,
 'nonce': 3}

In [85]:
# Send the ETH to an account in your testnet

send_tx(ETH, eth_acc,"0x268C8620F9a994410040D9B6DBE1E203451542c8", 18092513943330655534932)

0x36a1d69b4b0243d6f37f35a22b6c9ab8e7ee083d68b562f1f4dede48dac750cb


'0x36a1d69b4b0243d6f37f35a22b6c9ab8e7ee083d68b562f1f4dede48dac750cb'

In [88]:
# Check the balance of the recipient wallet. Bear in mind the gas fee. This matches the final screenshot (decimal point not withstanding)
w3.eth.getBalance("0x268C8620F9a994410040D9B6DBE1E203451542c8")


18092638331387208470032